In [18]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re
import numpy as np

import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from datetime import datetime

import os

In [19]:
class KIIDFound(Exception):
    pass

In [20]:
def find_kiids(pdf_links, pdf_links_titles, wzor1, wzor2):
    wynik=[]
    for i in range(len(pdf_links)):
        if pdf_links_titles[i] is not None:
            link_title = pdf_links_titles[i]
        else:
            link_title = ""
        ratio1 = fuzz.partial_ratio(pdf_links[i].lower(), wzor1)
        ratio2 = fuzz.partial_ratio(pdf_links[i].lower(), wzor2)
        ratio3 = fuzz.partial_ratio(link_title.lower(), wzor1)
        ratio4 = fuzz.partial_ratio(link_title.lower(), wzor2)

        if ratio1 > 80 or ratio3 > 80 or ratio2 > 95 or ratio4 > 95:

            wynik.append(pdf_links[i])
    return wynik
wzor1 = 'kluczowe informacje dla inwestorow'
wzor2 = 'kiid'
wzor3 = 'kluczowe-informacje-dla-inwestorow'

In [21]:
def save_pdf(url):
    r = requests.get(url, stream=True)
    domain = url.split(".")[1]
    if not os.path.exists(domain):
        os.makedirs(domain, exist_ok=True) 
    nazwa = domain +'/' + url.split("/")[-1]
    with open(nazwa, 'wb') as f:
        f.write(r.content)

In [35]:
def scrap_all_kiids(url, visited: list):
    def scrape_kiid(url, visited: list):
        if len(visited)>0 and url in visited:
            return
        visited.append(url)
        try:
            response = requests.get(url)
        except:
            return
        print(url)
        try:
            soup = BeautifulSoup(response.text, "html.parser")
        except:
            return
        hrefs = soup.findAll('a', href=True)
        titles = [hrefs.string for hrefs in hrefs]
        res = {}
        for key in hrefs:
            for value in titles:
                res[key["href"]] = value
                titles.remove(value)
                break
        
        array = np.unique([a["href"] for a in hrefs])
        if len(array) == 0:
            return
        no_https = array[~np.char.startswith(array, "http")]
        pdf_links = no_https[np.char.endswith(no_https, "pdf")].tolist()
        pdf_links_titles = list(map(res.get, pdf_links))
        no_https = no_https[~np.char.endswith(no_https, "pdf")]
        inner_site_links = no_https[np.char.startswith(no_https, "/")] #tu zmiana, żeby działało niezależnie od domeny
        #inner_site_links = no_https[np.char.endswith(no_https, "/")].tolist()
        inner_site_links = inner_site_links[~np.char.endswith(inner_site_links, "zip")].tolist()
        wynik = find_kiids(pdf_links, pdf_links_titles, wzor1 = 'kluczowe informacje dla inwestorow', wzor2 = 'kiid')
        if len(wynik) > 1:
            wynik_final.extend([domain[:-1]+wynik for wynik in wynik])
            raise KIIDFound
        #pdf_final_links = pdf_final_links + pdf_links
        #pdf_final_links.extend(pdf_links)
        visited.append(url)
        for link in inner_site_links:
            scrape_kiid(domain[:-1]+link, visited)
    try:
        scrape_kiid(url, visited)
    except KIIDFound:
        for pdf in wynik_final:
            save_pdf(pdf)
        print("Udalo sie!")


In [7]:
n = '''1 AgioFunds TFI S.A. www.agiofunds.pl
2 Alior TFI S.A. www.aliortfi.com
3 Amundi Polska TFI S.A. www.amundi.pl
4 Baltic Capital TFI S.A. www.baltictfi.pl
5 BNP Paribas TFI S.A. www.tfi.bnpparibas.pl
6 BPS TFI S.A. www.bpstfi.pl
7 Caspar TFI S.A. www.caspartfi.pl
8 EQUES INVESTMENT TFI S.A. www.eitfi.pl
9 Esaliens TFI S.A. www.esaliens.pl
10 Generali Investments TFI S.A. www.generali-investments.pl
11 Investors TFI S.A. www.investors.pl
12 Ipopema TFI S.A. www.ipopema.pl
13 MILLENNIUM TFI S.A. www.millenniumtfi.pl
14 Nationale-Nederlanden TFI S.A. www.metlife.pl
15 NN Investment Partners TFI S.A. www.nntfi.pl
16 Noble Funds TFI S.A. www.noblefunds.pl
17 OPERA TFI S.A. www.opera-tfi.pl
18 Pekao TFI S.A. www.pekaotfi.pl
19 PFR TFI S.A. www.pfrtfi.pl
20 PKO TFI S.A. www.pkotfi.pl
21 Quercus TFI S.A. www.qtfi.pl
22 Rockbridge TFI S.A. www.rockbridge.pl
23 Santander TFI S.A. www.santandertfi.pl
24 SKARBIEC TFI S.A. www.skarbiec.pl
25 Superfund TFI S.A. www.superfund.pl
26 Templeton Asset Management (Poland) TFI S.A. www.franklintempleton.pl
27 TFI AGRO S.A. www.tfiagro.pl
28 TFI Allianz Polska S.A. www.allianz.pl
29 TFI Energia S.A. www.tfienergia.pl
4
30 TFI PZU SA www.pzu.pl
31 Uniqa TFI S.A. www.uniqa.pl
32 VIG/C-QUADRAT TFI S.A. www.vigcq-tfi.p'''

In [8]:
n = n.split()
nazwy = np.array(n)
domains = nazwy[np.char.startswith(nazwy,'www')]
domains = ["https://" + i +'/' for i in domains]

In [32]:
domains = ['https://www.amundi.pl/',
 'https://www.baltictfi.pl/',
 'https://www.tfi.bnpparibas.pl/',
 'https://www.bpstfi.pl/',
 'https://www.caspar.com.pl/',
 'https://www.eitfi.pl/',
 'https://www.esaliens.pl/',
 'https://www.generali-investments.pl/',
 'https://www.investors.pl/',
 'https://www.ipopema.pl/',
 'https://www.millenniumtfi.pl/',
 'https://www.nn.pl/',
 'https://www.nntfi.pl/',
 'https://www.noblefunds.pl/',
 'http://www.opera.pl/',
 'https://www.pekaotfi.pl/',
 'https://www.pfrtfi.pl/',
 'https://www.pkotfi.pl/',
 'https://www.qtfi.pl/',
 'https://www.rockbridge.pl/',
 'https://www.santander.pl/tfi/',
 'https://www.skarbiec.pl/',
 'https://www.superfund.pl/',
 'https://www.franklintempleton.pl/',
 'https://www.tfiagro.pl/',
 'https://www.allianz.pl/',
 'https://www.tfienergia.pl/',
 'https://www.pzu.pl/',
 'https://www.uniqa.pl/',
 'https://www.vigcq-tfi.p/']

In [56]:
domains

'https://www.amundi.pl/'

In [51]:
domains = domains[0]

In [52]:
def scrapp_pages(domains):
    for domain in domains:
        visited = []
        wynik_final = []
        start = datetime.now()
        scrap_all_kiids(domain, visited)
        end = datetime.now()
        #print(domain + ": " + str(end-start))
        #print(wynik_final)

In [53]:
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count

In [54]:
def multi_scrapp(domains, scrapp_pages):
    pool = Pool(cpu_count())
    results = pool.map(scrapp_pages, domains)
    pool.close()
    pool.join()
    return results

In [57]:
multi_scrapp(domains, scrapp_pages)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]